In [3]:
import sys
import logging
import json
import os, time  #import statements
from datetime import datetime
import mysql.connector
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from stat import *
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import StringType
import pyspark.sql.functions as f


startTime = datetime.now()

import datetime
ts = time.time()
currentTimeStamp= datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

logging.basicConfig(filename='Ingestion_'+currentTimeStamp+'.log', filemode='w',format='%(asctime)s - %(message)s',datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)
logging.info('Job Started : %s',currentTimeStamp)
SparkContext.setSystemProperty("hive.metastore.uris", "thrift://10.3.2.20:9083")
sparkSession = (SparkSession.builder.appName('pyspark-to-load-tables-hive').enableHiveSupport().getOrCreate())
spark = SparkSession.builder.appName('changeColNames').getOrCreate()
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
logging.info('User logged in and created SparkContext and SparkSession')


feedname = sys.argv[1]
filename = sys.argv[2]
logging.info('Feteched command line arguments %s and %s',feedname,filename)
try:
    selectSql = 'SELECT * FROM etl.feedcontrol' + ' WHERE feedname' + " ='" + feedname + "' AND filename" + " ='" + filename +"'"
    logging.info('%s',selectSql)
    feed = sparkSession.sql(selectSql)
    feed.show()
    cal = sparkSession.sql('SELECT * FROM etl.calender where openindicator ="Y" ')  #calender table
    cal.show()
    filemetadata_selectSql = 'SELECT * FROM etl.filemetadata' + ' WHERE feedname' + " ='" + feedname + "' AND filename" + " ='" + filename +"'"
    filemetadata = sparkSession.sql(filemetadata_selectSql)
    filemetadata.show()
    try:
        df_ATracker = sparkSession.sql('SELECT max(attendenceid) as attendence_id FROM ETL.Attendence_Tracker order by attendence_id ')  #Attendence_tracker
        attendenceid = df_ATracker.collect()[0]['attendence_id']
        attendence_id = int(attendenceid) + 1
        logging.info('value of attendence_id is %s', attendence_id)
    except Exception as e:
        logging.error('Error occured while creating dataframe for attendence_tarcker tables', exc_info=True)
    feedname = feed.collect()[0]['feedname']
    filename = feed.collect()[0]['filename']
    landingpath = feed.collect()[0]['landingpath']
    sourceTableName = feed.collect()[0]['sourcetablename']
    header_trailer_flag = feed.collect()[0]['header_trailer_flag']
    fileformat = feed.collect()[0]['fileformat']
    rawzonepath = feed.collect()[0]['rawzonepath']
    processType = feed.collect()[0]['processtype']
    busdate_calender = cal.collect()[0]['busdate']
    if(fileformat == 'DELIMITED' or fileformat == 'FIXED'):
        filedelimiter = filemetadata.collect()[0]['filedelimiter']
    if(fileformat == 'database'):
        jsonpath = "/home/developer/Vedashree/MysqlTestTable.json"
    if(fileformat == 'DELIMITED' or 'FIXED'):
        path = str("%s/%s" % (landingpath, filename))
        print(filename)
    else:
        path = str("%s/%s%s%s" % (landingpath, filename,".",fileformat))
    rawzonepath = str("%s%s" % (rawzonepath, filename))
    print(rawzonepath)
    
    if(fileformat == 'database'):
        ArrivalTimeStamp=currentTimeStamp
        logging.info('ArrivalTimestamp for MysqlDB fileformat %s',ArrivalTimeStamp)
    else:                                                                                       #extracting arrival time stamp of file from system
        ArrivalTimeStamp = time.ctime(os.path.getctime(path))
    if(fileformat == 'database'):  #extracting file size
        FileSize = 0
    else :
        st = os.stat(path)
        FileSize = st[ST_SIZE]
except Exception as e:
    logging.info('Job is FAILED')
    logging.error('Error occured while creating dataframe or extracting data from feedcontrol and calender tables', exc_info=True)

 